## PREPROCESSING
### Get the dataset
Load the dataset, split it in two for trainin and validation. As in the Reference model provided by [Lyft](https://level5.lyft.com/), a dataframe with one scene per row is created.

In [2]:
DATASET_VERSION = 'v1.02-train'
DATASET_ROOT = '../../nuscenes-devkit/data/'

#The code will generate data, visualization and model checkpoints
ARTIFACTS_FOLDER = "./artifacts"

In [3]:
%load_ext autoreload

from datetime import datetime
from functools import partial
import glob
#Disabled for numpy and opencv: avod has opencv and numpy versions for several methods
from multiprocessing import Pool

import os
#os.environ["OMP_NUM_THREADS"] = "1"

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pandas as pd
import cv2
import PIL
from PIL import Image
import numpy as np
from tqdm import tqdm, tqdm_notebook
import argparse
import scipy
import scipy.ndimage
import scipy.special
from scipy.spatial.transform import Rotation as R

from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix


In [4]:
gpus = tf.compat.v1.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.4) 
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True, gpu_options=gpu_options))

1 Physical GPUs, 1 Logical GPU
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 850M, pci bus id: 0000:0a:00.0, compute capability: 5.0
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [5]:
level5data = LyftDataset(json_path=DATASET_ROOT + "/v1.02-train", data_path=DATASET_ROOT, verbose=True)
os.makedirs(ARTIFACTS_FOLDER, exist_ok=True)

9 category,
18 attribute,
4 visibility,
15991 instance,
8 sensor,
128 calibrated_sensor,
149072 ego_pose,
148 log,
148 scene,
18634 sample,
149072 sample_data,
539765 sample_annotation,
1 map,
Done loading in 11.4 seconds.
Reverse indexing ...
Done reverse indexing in 3.8 seconds.


## <font color="pink">TRAINING</font>
Based on the [AVOD algorithm](https://github.com/kujason/avod), we train the dataset. 
The goal is to study how the accuracy changes based on the type of sensors in input, and their number, thus changes to the AVOD algorithm have been made. Here we keep the two stage model.
Will be divided in steps, to mimick the divisions made by AVOD's authors in the code.

With respect to the original AVOD code, the following changes have been made:
<li> Upgrades for compatibity issues with tensorflow 2.0: migrated from slim libs to keras Sequential</li>
<li> Changes to support single type input </li>
<li> VGGs take as input Lyft-style dataset </li>



In [6]:
import avod
from avod.core import trainer

<b>RPN MODEL</b>: It is the fist subnetwork that makes up the double stage AVOD algorithm. It uses two VGGs, one for images, one for LiDar, to find the bottleneck.
Img VGG and Bev VGG have the same strucure, just have input from different sources. 

<b>VGG:</b> VGG is a convolutional neural network model. Here simplified model wrt K. Simonyan and A. Zisserman's model proposed in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition".
Basically, it lacks dense layers at the end, and the last group of conv layers is smaller that theirs.
Two VGGs, one for BEV, one for Images

In [7]:
import anchor_helper
from frame_helper import FrameCalibrationData
import frame_helper
import preproc_helper
import bev_helper

In [8]:
from PIL import Image
from avod.core import anchor_filter
from avod.core import anchor_projector
from avod.core import box_3d_encoder
from avod.core import constants
from avod.core import losses
from avod.core import model
from avod.core import summary_utils
from avod.core.anchor_generators import grid_anchor_3d_generator
from avod.datasets.kitti import kitti_aug
import avod.datasets.kitti.kitti_utils as kitti_utils
from avod.core.label_cluster_utils import LabelClusterUtils

In [9]:
class RpnModel(model.DetectionModel):
    ##############################
    # Keys for Placeholders
    ##############################
    PL_BEV_INPUT = 'bev_input_pl'
    PL_IMG_INPUT = 'img_input_pl'
    PL_ANCHORS = 'anchors_pl'

    PL_BEV_ANCHORS = 'bev_anchors_pl'
    PL_BEV_ANCHORS_NORM = 'bev_anchors_norm_pl'
    PL_IMG_ANCHORS = 'img_anchors_pl'
    PL_IMG_ANCHORS_NORM = 'img_anchors_norm_pl'
    PL_LABEL_ANCHORS = 'label_anchors_pl'
    PL_LABEL_BOXES_3D = 'label_boxes_3d_pl'
    PL_LABEL_CLASSES = 'label_classes_pl'

    PL_ANCHOR_IOUS = 'anchor_ious_pl'
    PL_ANCHOR_OFFSETS = 'anchor_offsets_pl'
    PL_ANCHOR_CLASSES = 'anchor_classes_pl'

    # Sample info, including keys for projection to image space
    # (e.g. camera matrix, image index, etc.)
    PL_CALIB_P2 = 'frame_calib_p2'
    PL_IMG_IDX = 'current_img_idx'
    PL_GROUND_PLANE = 'ground_plane'

    ##############################
    # Keys for Predictions
    ##############################
    PRED_ANCHORS = 'rpn_anchors'

    PRED_MB_OBJECTNESS_GT = 'rpn_mb_objectness_gt'
    PRED_MB_OFFSETS_GT = 'rpn_mb_offsets_gt'

    PRED_MB_MASK = 'rpn_mb_mask'
    PRED_MB_OBJECTNESS = 'rpn_mb_objectness'
    PRED_MB_OFFSETS = 'rpn_mb_offsets'

    PRED_TOP_INDICES = 'rpn_top_indices'
    PRED_TOP_ANCHORS = 'rpn_top_anchors'
    PRED_TOP_OBJECTNESS_SOFTMAX = 'rpn_top_objectness_softmax'

    ##############################
    # Keys for Loss
    ##############################
    LOSS_RPN_OBJECTNESS = 'rpn_objectness_loss'
    LOSS_RPN_REGRESSION = 'rpn_regression_loss'

    def __init__(self, model_config, pipeline_config, train_val_test, dataset):
        """
        Args:
            model_config: configuration for the model
            train_val_test: "train", "val", or "test"
            dataset: the dataset that will provide samples and ground truth
        """

        # Sets model configs (_config)
        super(RpnModel, self).__init__(model_config)
        self.dataset=dataset
        self.pipeline_config = pipeline_config

        if train_val_test not in ["train", "val", "test"]:
            raise ValueError('Invalid train_val_test value,'
                             'should be one of ["train", "val", "test"]')
        self._train_val_test = train_val_test

        self._is_training = (self._train_val_test == 'train')

        # Input config
        input_config = self._config.input_config
        self._bev_pixel_size = np.asarray([input_config.bev_dims_h, input_config.bev_dims_w])
        self._bev_depth = input_config.bev_depth
        self._img_pixel_size = np.asarray([input_config.img_dims_h, input_config.img_dims_w])
        self._img_depth = input_config.img_depth

        # Rpn config
        rpn_config = self._config.rpn_config
        self.proposal_roi_crop_size = 3*2  #3*2
        self._fusion_method = rpn_config.rpn_fusion_method
        if self._train_val_test in ["train", "val"]:
            self._nms_size = rpn_config.rpn_train_nms_size
        else:
            self._nms_size = rpn_config.rpn_test_nms_size

        self._nms_iou_thresh = rpn_config.rpn_nms_iou_thresh

#         # Network input placeholders
#         self.placeholders = dict()
#         # Inputs to network placeholders
#         self._placeholder_inputs = dict()
#         # Information about the current sample
#         self.sample_info = dict()

        # Dataset
        classes=[]
        for i in self.dataset.category:
            classes.append(i.get("name"))
        self.classes = classes
        self.dataset = dataset
        self.dataset.train_val_test = self._train_val_test
        area_extents = self.pipeline_config.kitti_utils_config.area_extents
        self._area_extents = np.reshape(area_extents, (3, 2))
        self._bev_extents = self._area_extents[[0, 2]]
        
        label_cluster_utils = LabelClusterUtils(self.dataset)
        self._cluster_sizes, self._all_std = label_cluster_utils.get_clusters(5, self.dataset)
        
        anchor_strides = self.pipeline_config.kitti_utils_config.anchor_strides
        self._anchor_strides= np.reshape(anchor_strides, (-1, 2))
        self._anchor_generator = grid_anchor_3d_generator.GridAnchor3dGenerator()

        self._path_drop_probabilities = self._config.path_drop_probabilities
        self._train_on_all_samples = self._config.train_on_all_samples
        self._eval_all_samples = self._config.eval_all_samples
        
        # Inputs-initialize empty, call method to fill. 
        self.img_input=[]
        self.bev_input=[]
        
        # Anchors
        self._anchors=[]
        self._anchor_offsets=[]
        self._anchor_ious=[]
        self._anchor_classes=[]
        self._bev_anchors=[]
        self._img_anchors=[]
        self._bev_anchors_norm=[]
        self._img_anchors_norm=[]
        
        # Labels
        self._label_anchors=[]
        self._label_classes=[]
        self._label_boxes_3d=[]
        
        # Other stuff
        self._sample_name=[]
        self._calib=[]
        self._ground_plane=[]
        

        if self._train_val_test in ["val", "test"]:
            # Disable path-drop, this should already be disabled inside the
            # evaluator, but just in case.
            self._path_drop_probabilities[0] = 1.0
            self._path_drop_probabilities[1] = 1.0

    def _add_placeholder(self, dtype, shape, name):
        placeholder = tf.compat.v1.placeholder(dtype, shape, name)
        self.placeholders[name] = placeholder
        return placeholder

    def _set_up_input_pls(self):
        """Sets up input placeholders by adding them to self._placeholders.
        Keys are defined as self.PL_*.
        """
        # Combine config data
        bev_dims = np.append(self._bev_pixel_size, self._bev_depth)
        

        with tf.compat.v1.variable_scope('bev_input'):
            # Placeholder for BEV image input, to be filled in with feed_dict
            bev_input_placeholder = self._add_placeholder(tf.float32, bev_dims,
                                                          self.PL_BEV_INPUT)

            self._bev_input_batches = tf.expand_dims(
                bev_input_placeholder, axis=0)

            self._bev_preprocessed = tf.image.resize(self._bev_input_batches, self._bev_pixel_size)

            # Summary Images
            bev_summary_images = tf.split(bev_input_placeholder, self._bev_depth, axis=2)
            tf.summary.image("bev_maps", bev_summary_images, max_outputs=self._bev_depth)

        with tf.compat.v1.variable_scope('img_input'):
            # Take variable size input images
            img_input_placeholder = self._add_placeholder(tf.float32, [None, None, self._img_depth],self.PL_IMG_INPUT)

            self._img_input_batches = tf.expand_dims(img_input_placeholder, axis=0)

            self._img_preprocessed = tf.image.resize(self._img_input_batches, self._img_pixel_size)

            # Summary Image
            tf.summary.image("rgb_image", self._img_preprocessed, max_outputs=2)

        with tf.compat.v1.variable_scope('pl_labels'):
            self._add_placeholder(tf.float32, [None, 6], self.PL_LABEL_ANCHORS)
            self._add_placeholder(tf.float32, [None, 7], self.PL_LABEL_BOXES_3D)
            #self._add_placeholder(tf.float32, [None], self.PL_LABEL_CLASSES)

        # Placeholders for anchors
#         with tf.compat.v1.variable_scope('pl_anchors'):
#             self._add_placeholder(tf.float32, [None, 6], self.PL_ANCHORS)
#             self._add_placeholder(tf.float32, [None], self.PL_ANCHOR_IOUS)
#             self._add_placeholder(tf.float32, [None, 6], self.PL_ANCHOR_OFFSETS)
#             self._add_placeholder(tf.float32, [None], self.PL_ANCHOR_CLASSES)

#             with tf.compat.v1.variable_scope('bev_anchor_projections'):
#                 self._add_placeholder(tf.float32, [None, 4], self.PL_BEV_ANCHORS)
#                 self._bev_anchors_norm_pl = self._add_placeholder( tf.float32, [None, 4], self.PL_BEV_ANCHORS_NORM)

#             with tf.compat.v1.variable_scope('img_anchor_projections'):
#                 self._add_placeholder(tf.float32, [None, 4], self.PL_IMG_ANCHORS)
#                 self._img_anchors_norm_pl = self._add_placeholder( tf.float32, [None, 4], self.PL_IMG_ANCHORS_NORM)

#             with tf.compat.v1.variable_scope('sample_info'):
#                 # the calib matrix shape is (3 x 4)
#                 self._add_placeholder( tf.float32, [3, 4], self.PL_CALIB_P2)
#                 self._add_placeholder(tf.int32, shape=[1], name=self.PL_IMG_IDX)
#                 self._add_placeholder(tf.float32, [4], self.PL_GROUND_PLANE)
        

    def build(self):
        #call here fill input??
        self.feed_input(5)
        # Setup feature extractors: two models, one for each kind of input
        weight_decay=0.0005
        #shape due to shape provided by dataset. BEV could not be adapted: too sparse.
        
        print("\nStarting with the extractors..\n")
        with tf.device('/CPU:0'):
            inputs_vgg = tf.keras.layers.Input(shape=(224,224,3))
            vgg19 = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
            bev_data = tf.keras.applications.vgg19.preprocess_input(self.bev_input)
            bev_features=vgg19.predict(bev_data)
            #struggle with memory
            img_data = tf.keras.applications.vgg19.preprocess_input(self.img_input)
            img_features=vgg19.predict(img_data)

            bev_proposal_input = self.bev_bottleneck
            img_proposal_input = self.img_bottleneck

            fusion_mean_div_factor = 2.0
        print("\nFINISHED FEATURE EXTRACTION\n")
        
        # Select what to keep btw imgs and bev
        if not (self._path_drop_probabilities[0] ==
                self._path_drop_probabilities[1] == 1.0):
            with tf.compat.v1.variable_scope('rpn_path_drop'):

                random_values = tf.random_uniform(shape=[3], minval=0.0, maxval=1.0)
                img_mask, bev_mask = self.create_path_drop_masks(self._path_drop_probabilities[0], self._path_drop_probabilities[1], random_values)
                img_proposal_input = tf.multiply(img_proposal_input, img_mask)
                bev_proposal_input = tf.multiply(bev_proposal_input, bev_mask)
                self.img_path_drop_mask = img_mask
                self.bev_path_drop_mask = bev_mask

                # Overwrite the division factor
                fusion_mean_div_factor = img_mask + bev_mask
        print("\nStarting ROI pooling..\n")
        
        with tf.compat.v1.variable_scope('proposal_roi_pooling'):

            with tf.compat.v1.variable_scope('box_indices'):
                def get_box_indices(boxes):
                    proposals_shape = boxes.get_shape().as_list()
                    if any(dim is None for dim in proposals_shape):
                        proposals_shape = tf.shape(boxes)
                    ones_mat = tf.ones(proposals_shape[:2], dtype=tf.int32)
                    multiplier = tf.expand_dims(tf.range(start=0, limit=proposals_shape[0]), 1)
                    return tf.reshape(ones_mat * multiplier, [-1])

                bev_boxes_norm_batches = tf.expand_dims(self._bev_anchors_norm, axis=0)
                tf_box_indices = get_box_indices(bev_boxes_norm_batches)
            
            proposal_roi_size_tf = [3,3]
            # Do ROI Pooling on BEV eager ex on _bev_ancho
            bev_proposal_rois = tf.image.crop_and_resize(bev_proposal_input,
                                                         self._bev_anchors_norm, tf_box_indices, proposal_roi_size_tf)
            # Do ROI Pooling on image
            img_proposal_rois = tf.image.crop_and_resize(img_proposal_input,
                                                         self._img_anchors_norm, tf_box_indices, proposal_roi_size_tf)

        with tf.compat.v1.variable_scope('proposal_roi_fusion'):
            rpn_fusion_out = None
            if self._fusion_method == 'mean':
                tf_features_sum = tf.add(bev_proposal_rois, img_proposal_rois)
                rpn_fusion_out = tf.divide(tf_features_sum, 2)
            elif self._fusion_method == 'concat':
                rpn_fusion_out = tf.concat([bev_proposal_rois, img_proposal_rois], axis=3)
            else:
                raise ValueError('Invalid fusion method', self._fusion_method)

        with tf.compat.v1.variable_scope('anchor_predictor', 'ap', [rpn_fusion_out]):
            
            print("here", tf_features_sum)
            # Rpn layers config
            weight_decay = 0.005
            
            tensor_in = tf.keras.Input(shape=None, tensor=rpn_fusion_out)

            # Use conv2d instead of fully_connected layers.
            cls_fc6 = tf.keras.layers.Conv2D(filters=32, kernel_size = [3,3], kernel_initializer='ones', 
                                             kernel_regularizer=tf.keras.regularizers.l2(weight_decay), padding='VALID', name="conv1")(tensor_in)
            cls_fc6_drop = tf.keras.layers.Dropout(rate = 0.5, name="drop1")(cls_fc6)
            cls_fc7 = tf.keras.layers.Conv2D(filters=32, kernel_size = [1,1], kernel_initializer='ones',
                                             kernel_regularizer=tf.keras.regularizers.l2(weight_decay), padding='VALID', name="conv2")(cls_fc6_drop)
            cls_fc7_drop = tf.keras.layers.Dropout(rate = 0.5, name="drop2")(cls_fc7)
            cls_fc8 = tf.keras.layers.Conv2D(filters=2, kernel_size = [1,1], kernel_initializer='ones',
                                             kernel_regularizer=tf.keras.regularizers.l2(weight_decay), padding='VALID', name="conv3")(cls_fc7_drop)
            objectness = tf.squeeze(cls_fc8, axis=[1,2], name='conv3/squeezed')

            obj_model = tf.keras.models.Model(inputs = rpn_fusion_out, outputs = objectness, name="objectness predictions")
            obj_model.summary()
            bev_vgg.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])
            bev_hist = bev_vgg.fit(self.input_bev, batch_size=64, epochs=50, validation_split=0.2)

            # Use conv2d instead of fully_connected layers.
            reg_fc6 = tf.keras.layers.Conv2D(filters=32, kernel_size = [3,3], kernel_initializer="ones", 
                                    kernel_regularizer=tf.keras.regularizers.l2(weight_decay), padding='VALID', name="conv4")(tensor_in)

            reg_fc6_drop = tf.keras.layers.Dropout(rate = 0.5, name="drop3")(reg_fc6)

            reg_fc7 = tf.keras.layers.Conv2D(filters = 16, kernel_size = [1, 1], kernel_initializer="ones",
                                    kernel_regularizer=tf.keras.regularizers.l2(weight_decay), padding="same", name="conv5")(reg_fc6_drop)

            reg_fc7_drop = tf.keras.layers.Dropout(rate = 0.5, name="drop4")(reg_fc7)

            reg_fc8 = tf.keras.layers.Conv2D(filters = 6,  kernel_size = [1, 1],  kernel_initializer="ones",
                                    kernel_regularizer=tf.keras.regularizers.l2(weight_decay), padding="same", name="conv6")(reg_fc7_drop)

            offsets = tf.squeeze(reg_fc8, axis=[1,2], name='conv6/squeezed')
            
            off_model = tf.keras.models.Model(inputs = rpn_fusion_out, outputs = offsets, name="rpn_fusion_prediction_anchors")
            off_model.summary()
            bev_vgg.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=["accuracy"])
            bev_hist = bev_vgg.fit(self.input_bev, batch_size=64, epochs=50, validation_split=0.2)
            
            
        # Return the proposals
        with tf.compat.v1.variable_scope('proposals'):
            #anchors = self.placeholders[self.PL_ANCHORS]
            anchors = self._anchors

            # Decode anchor regression offsets
            with tf.compat.v1.variable_scope('decoding'):
                regressed_anchors = anchor_helper.offset_to_anchor(anchors, offsets)

            with tf.compat.v1.variable_scope('bev_projection'):
                _, bev_proposal_boxes_norm = anchor_projector.project_to_bev(regressed_anchors, self._bev_extents)

            with tf.compat.v1.variable_scope('softmax'):
                objectness_softmax = tf.nn.softmax(objectness)

            with tf.compat.v1.variable_scope('nms'):
                objectness_scores = objectness_softmax[:, 1]

                # Do NMS on regressed anchors
                top_indices = tf.image.non_max_suppression(
                    bev_proposal_boxes_norm, objectness_scores,
                    max_output_size=self._nms_size,
                    iou_threshold=self._nms_iou_thresh)

                top_anchors = tf.gather(regressed_anchors, top_indices)
                top_objectness_softmax = tf.gather(objectness_scores,
                                                   top_indices)
                top_offsets = tf.gather(offsets, top_indices)
                top_objectness = tf.gather(objectness, top_indices)

        # Get mini batch #TODO eager execution 
        #all_ious_gt = self.placeholders[self.PL_ANCHOR_IOUS]
        #all_offsets_gt = self.placeholders[self.PL_ANCHOR_OFFSETS]
        #all_classes_gt = self.placeholders[self.PL_ANCHOR_CLASSES]
        all_offsets_gt =  self._anchor_offsets
        all_ious_gt = self._anchor_ios
        all_classes_gt = self._anchor_classes

        with tf.compat.v1.variable_scope('mini_batch'):
            mini_batch_mask, _ = anchor_helper.sample_mini_batch(all_ious_gt, 64,[0, 0.3], [0.5,1])

        # ROI summary images
        rpn_mini_batch_size = 64
        with tf.compat.v1.variable_scope('bev_rpn_rois'):
            mb_bev_anchors_norm = tf.boolean_mask(self._bev_anchors_norm, mini_batch_mask)
            mb_bev_box_indices = tf.zeros_like(tf.boolean_mask(all_classes_gt, mini_batch_mask), dtype=tf.int32)

            # Show the ROIs of the BEV input density map
            # for the mini batch anchors
            bev_input_rois = tf.image.crop_and_resize(self.bev_input, mb_bev_anchors_norm, mb_bev_box_indices, (32, 32))

            bev_input_roi_summary_images = tf.split(bev_input_rois, self._bev_depth, axis=3)
            tf.summary.image('bev_rpn_rois', bev_input_roi_summary_images[-1], max_outputs=rpn_mini_batch_size)

        with tf.compat.v1.variable_scope('img_rpn_rois'):
            # ROIs on image input TODO placeolder
            mb_img_anchors_norm = tf.boolean_mask(self._img_anchors_norm, mini_batch_mask)
            mb_img_box_indices = tf.zeros_like( tf.boolean_mask(all_classes_gt, mini_batch_mask), dtype=tf.int32)

            # Do test ROI pooling on mini batch
            img_input_rois = tf.image.crop_and_resize(self.img_input, mb_img_anchors_norm, mb_img_box_indices, (32, 32))
            tf.summary.image('img_rpn_rois', img_input_rois, max_outputs=rpn_mini_batch_size)

        # Ground Truth Tensors
        with tf.compat.v1.variable_scope('one_hot_classes'):

            # Anchor classification ground truth
            # Object / Not Object
            min_pos_iou = 0.5

            objectness_classes_gt = tf.cast(tf.greater_equal(all_ious_gt, min_pos_iou), dtype=tf.int32)
            objectness_gt = tf.one_hot(objectness_classes_gt, depth=2, on_value=1.0 - self._config.label_smoothing_epsilon,
                                       off_value=self._config.label_smoothing_epsilon)

        # Mask predictions for mini batch
        with tf.compat.v1.variable_scope('prediction_mini_batch'):
            objectness_masked = tf.boolean_mask(objectness, mini_batch_mask)
            offsets_masked = tf.boolean_mask(offsets, mini_batch_mask)

        with tf.compat.v1.variable_scope('ground_truth_mini_batch'):
            objectness_gt_masked = tf.boolean_mask(objectness_gt, mini_batch_mask)
            offsets_gt_masked = tf.boolean_mask(all_offsets_gt, mini_batch_mask)

        # Specify the tensors to evaluate
        predictions = dict()

        predictions[self.PRED_TOP_OBJECTNESS_SOFTMAX] = top_objectness_softmax


    def feed_input(self, scene_index=None):
        """ Fills in the placeholders with the actual input values.
            Currently, only a scene is supported

        Args:
            scene_idx: TODO
        Returns:
            a feed_dict dictionary that can be used in a tensorflow session
        """
        #TODO: read scenes up to scene_idx: modify preproc scipt accordingly

        if scene_index is not None:
            my_scene = self.dataset.scene[scene_index]
        else:
            raise TypeError('for testing you need to put a number! will change it later on once it works fully :) ')
    
        #general data.
        
        voxel_size = (0.4,0.4,1.5)
        z_offset = -2.0
        #arbitrary shape, must be square though!
        bev_shape = (336,336, 3)
        
        # Only handle one sample at a time for now
        my_sample_token = my_scene["first_sample_token"]
        last_sample_token = my_scene["last_sample_token"]

        while my_sample_token!=last_sample_token:

            # We only need orientation from box_3
            sample = self.dataset.get('sample', my_sample_token)
            sample_name = sample.get("token")
            anchors_info, obj_classes, label_classes, label_anchors, label_boxes_3d = preproc_helper.load_sample_info(sample_name, self.classes, self.dataset)

            # Network input data: loop to get batch info
            img_input = self.dataset.get('sample_data', sample['data']["CAM_FRONT"])
            img_data=img_input
            camera_token=img_input.get("token")
            file_name=self.dataset.get_sample_data_path(camera_token)
            image = Image.open(file_name)
            # convert image to numpy array
            img_array = np.asarray(image)
            self.img_input.append(img_array)
            # Image shape (h, w), fixed, dunno if need to save
            image_shape = [img_data.get("height"), img_data.get("width")]
            
            bev_input = self.dataset.get('sample_data', sample['data']["LIDAR_TOP"])
            bev_data = bev_input
            bev_token= bev_input.get("token")
            lidar_data = self.dataset.get("sample_data", bev_token)
            lidar_filepath = self.dataset.get_sample_data_path(bev_token)
            ego_pose = self.dataset.get("ego_pose", lidar_data["ego_pose_token"])
            calibrated_sensor_lidar = self.dataset.get("calibrated_sensor", lidar_data["calibrated_sensor_token"])
            global_from_car = transform_matrix(ego_pose['translation'], Quaternion(ego_pose['rotation']), inverse=False)
            car_from_sensor_lidar = transform_matrix(calibrated_sensor_lidar['translation'], Quaternion(calibrated_sensor_lidar['rotation']),
                                                      inverse=False)
            lidar_pointcloud = LidarPointCloud.from_file(lidar_filepath)
            lidar_pointcloud.transform(car_from_sensor_lidar)
            map_mask = level5data.map[0]["mask"]
            bev = bev_helper.create_voxel_pointcloud(lidar_pointcloud.points, bev_shape, voxel_size=voxel_size, z_offset=z_offset)
            ego_centric_map = bev_helper.get_semantic_map_around_ego(map_mask, ego_pose, voxel_size=0.4, output_shape=(336,336)) 
            bev_array = bev_helper.normalize_voxel_intensities(bev)
            self.bev_input.append(bev_array)

            #ground plane shape (a,b,c,d) in kitti:
            #no info on ground plane in nuscenes data, just global coordinate system
            #which is given as x, y, z. Computed from the cameras position:
            #suppose as in kitti that ground plane is as the same level with the cameras

            cam_front_token = self.dataset.get('sample_data', sample['data']["CAM_FRONT"])
            cam_front_data = cam_front_token.get("calibrated_sensor_token")
            cam_front_calib = self.dataset.get("calibrated_sensor", cam_front_data )
            cam_front_coords = cam_front_calib.get("translation")

            cam_front_left_token = self.dataset.get('sample_data', sample['data']["CAM_FRONT_LEFT"])
            cam_front_left_data = cam_front_left_token.get("calibrated_sensor_token")
            cam_front_left_calib = self.dataset.get("calibrated_sensor", cam_front_left_data )
            cam_front_left_coords = cam_front_left_calib.get("translation")

            cam_front_right_token = self.dataset.get('sample_data', sample['data']["CAM_FRONT_RIGHT"])
            cam_front_right_data = cam_front_right_token.get("calibrated_sensor_token")
            cam_front_right_calib = self.dataset.get("calibrated_sensor", cam_front_right_data )
            cam_front_right_coords = cam_front_right_calib.get("translation")

            ground_plane = frame_helper.get_ground_plane_coeff(cam_front_coords, cam_front_left_coords, cam_front_right_coords)
            
            #only for cameras, of course lidars do not have instrinsic matrices
            token=img_data.get("calibrated_sensor_token") 
            stereo_calib_p2 = frame_helper.read_calibration(token, self.dataset)
            
            #get next sample to extract info
            my_sample_token = self.dataset.get("sample", my_sample_token)["next"]

            # Fill the placeholders for anchor information
            self._fill_inputs(anchors_info=anchors_info,sample_token=bev_token, ground_plane=ground_plane,
                                        image_shape=image_shape, stereo_calib_p2=stereo_calib_p2,
                                        sample_name=sample_name)

            self._label_anchors.append(label_anchors)
            self._label_boxes_3d.append(label_boxes_3d)
            self._label_classes.append(label_classes)

            self._sample_name.append(str(sample_name))
            self._calib.append(stereo_calib_p2)
            self._ground_plane.append(ground_plane)
            
            return self.img_input

    def _fill_inputs(self,
                               sample_token,
                               anchors_info,
                               ground_plane,
                               image_shape,
                               stereo_calib_p2,
                               sample_name):
        """
        Fills anchor placeholder inputs with corresponding data

        Args:
            anchors_info: anchor info from mini_batch_utils
            ground_plane: ground plane coefficients
            image_shape: image shape (h, w), used for projecting anchors
            sample_name: name of the sample, e.g. "000001"
        """

        # Lists for merging anchors info
        all_anchor_boxes_3d = []
        anchors_ious = []
        anchor_offsets = []
        anchor_classes = []
        
        # Create anchors for each class
        if len(self.classes) > 1:
            for class_idx in range(len(self.classes)):
                cluster_sizes = []
                for i in self._cluster_sizes[class_idx]:
                    if i!=[]:
                        cluster_sizes.append(i)
                if len(cluster_sizes)!=0:
                # Generate anchors for all classes
                    grid_anchor_boxes_3d = self._anchor_generator.generate(
                        area_3d=self._area_extents,
                        anchor_3d_sizes=cluster_sizes,
                        anchor_stride=self._anchor_strides[0],
                        ground_plane=ground_plane)
                else:
                    #no labels per class, no anchor per class
                    grid_anchor_boxes_3d=[]
                all_anchor_boxes_3d.append(grid_anchor_boxes_3d)
            
            all_anchor_boxes_3d = np.concatenate(all_anchor_boxes_3d, axis=None)
        else:
            # Don't loop for a single class
            class_idx = 0
            cluster_sizes[class_idx] = [x for x in self._cluster_sizes[class_idx] if x != []]
            if self._cluster_sizes[class_idx]!=[]:
                grid_anchor_boxes_3d = self._anchor_generator.generate(area_3d=self._area_extents, anchor_3d_sizes=cluster_sizes[class_idx], 
                                                                       anchor_stride=self._anchor_strides[0], ground_plane=ground_plane)
                all_anchor_boxes_3d = grid_anchor_boxes_3d

        # Filter empty anchors
        # Skip if anchors_info is []
        sample_has_labels = True

        # Convert lists to ndarrays
        #already filtered them before
        anchor_boxes_3d_to_use = all_anchor_boxes_3d
        anchors_ious = np.asarray(anchors_ious)
        anchor_offsets = np.asarray(anchor_offsets)
        anchor_classes = np.asarray(anchor_classes)

        # Flip anchors and centroid x offsets for augmented samples
#             if kitti_aug.AUG_FLIPPING in sample_augs:
#                 anchor_boxes_3d_to_use = kitti_aug.flip_boxes_3d(anchor_boxes_3d_to_use, flip_ry=False)
#                 if anchors_info:
#                     anchor_offsets[:, 0] = -anchor_offsets[:, 0]

        # Convert to anchors
        anchors_to_use = box_3d_encoder.box_3d_to_anchor(anchor_boxes_3d_to_use)
        num_anchors = len(anchors_to_use)

        # Project anchors into bev
        bev_anchors, bev_anchors_norm = anchor_projector.project_to_bev( anchors_to_use, self._bev_extents)

        # Project box_3d anchors into image space
        img_anchors, img_anchors_norm = anchor_projector.project_to_image_space(anchors_to_use, stereo_calib_p2, image_shape)

        # Reorder into [y1, x1, y2, x2] for tf.crop_and_resize op
        self._bev_anchors_norm.append(bev_anchors_norm[:, [1, 0, 3, 2]])
        self._img_anchors_norm.append(img_anchors_norm[:, [1, 0, 3, 2]])

        # Fill in placeholder inputs
        self._anchors.append(anchors_to_use)
        #self._placeholder_inputs[self.PL_ANCHORS] = anchors_to_use

        # If we are in train/validation mode, and the anchor infos
        # are not empty, store them. Checking for just anchors_ious
        # to be non-empty should be enough.
        if self._train_val_test in ['train', 'val'] and len(anchors_ious) > 0:
            self._anchor_ious.append(anchors_ious)
            self._anchor_offsets.append(anchor_offsets)
            self._anchor_classes.append(anchor_classes)

        # During test, or val when there is no anchor info
        elif self._train_val_test in ['test'] or len(anchors_ious) == 0:
            # During testing, or validation with no gt, fill these in with 0s
#             self._placeholder_inputs[self.PL_ANCHOR_IOUS] = np.zeros(num_anchors)
#             self._placeholder_inputs[self.PL_ANCHOR_OFFSETS] = np.zeros([num_anchors, 6])
#             self._placeholder_inputs[self.PL_ANCHOR_CLASSES] = np.zeros(num_anchors)
            self._anchor_ious.append(anchors_ious)
            self._anchor_offsets.append(anchor_offsets)
            self._anchor_classes.append(anchor_classes)
                                                      
        else:
            raise ValueError('Got run mode {}, and non-empty anchor info'.format(self._train_val_test))

        self._bev_anchors = bev_anchors
        self._img_anchors = img_anchors
                                                      

    def create_path_drop_masks(self,
                               p_img,
                               p_bev,
                               random_values):
        """Determines global path drop decision based on given probabilities.

        Args:
            p_img: A tensor of float32, probability of keeping image branch
            p_bev: A tensor of float32, probability of keeping bev branch
            random_values: A tensor of float32 of shape [3], the results
                of coin flips, values should range from 0.0 - 1.0.

        Returns:
            final_img_mask: A constant tensor mask containing either one or zero
                depending on the final coin flip probability.
            final_bev_mask: A constant tensor mask containing either one or zero
                depending on the final coin flip probability.
        """

        def keep_branch(): return tf.constant(1.0)

        def kill_branch(): return tf.constant(0.0)

        # The logic works as follows:
        # We have flipped 3 coins, first determines the chance of keeping
        # the image branch, second determines keeping bev branch, the third
        # makes the final decision in the case where both branches were killed
        # off, otherwise the initial img and bev chances are kept.

        img_chances = tf.case([(tf.less(random_values[0], p_img), keep_branch)], default=kill_branch)

        bev_chances = tf.case([(tf.less(random_values[1], p_bev), keep_branch)], default=kill_branch)

        # Decision to determine whether both branches were killed off
        third_flip = tf.logical_or(tf.cast(img_chances, dtype=tf.bool), tf.cast(bev_chances, dtype=tf.bool))
        third_flip = tf.cast(third_flip, dtype=tf.float32)

        # Make a second choice, for the third case
        # Here we use a 50/50 chance to keep either image or bev
        # If its greater than 0.5, keep the image
        img_second_flip = tf.case([(tf.greater(random_values[2], 0.5), keep_branch)], default=kill_branch)
        # If its less than or equal to 0.5, keep bev
        bev_second_flip = tf.case([(tf.less_equal(random_values[2], 0.5), keep_branch)],
                                  default=kill_branch)

        # Use lambda since this returns another condition and it needs to
        # be callable
        final_img_mask = tf.case([(tf.equal(third_flip, 1), lambda: img_chances)], default=lambda: img_second_flip)

        final_bev_mask = tf.case([(tf.equal(third_flip, 1), lambda: bev_chances)], default=lambda: bev_second_flip)

        return final_img_mask, final_bev_mask


## Test this part!!

Tests and results for the RPN model part. The following changes to the model were done after errors/issues with testing:
<li>Maybe change the use of placeholders in the future, to fit with eager execution (shorter code)</li>
<li>Some internal keras os function rises a warning, something will be depreated, doesn't tell where and what function exactly</li>

In [11]:
import avod.builders.config_builder_util as config_build
config_path = 'avod/configs/unittest_model.config'
pipe_path = 'avod/configs/unittest_pipeline.config'
model_config = config_build.get_model_config_from_file(config_path)
pipeline_config=config_build.get_configs_from_pipeline_file(pipe_path, "val")

#rpn_model = RpnModel(model_config, pipeline_config[3],
#                          train_val_test="val",
#                          dataset=level5data)
# array=rpn_model.feed_input(5)
#predictions = rpn_model.build()
#tensor=tf.convert_to_tensor(array)


<b> AVOD MODEL: </b> second stage detector for the AVOD algorithm. It uses FPN as feature extractors.
<b> FPN: </b> Feature Pyramid Network (FPN) is a feature extractor designed for such pyramid concept with accuracy and speed in mind. It replaces the feature extractor of detectors like Faster R-CNN and generates multiple feature map layers (multi-scale feature maps) with better quality information than the regular feature pyramid for object detection. [Understanding Feature Pyramid Networks for object detection (FPN)](https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)

In [ ]:
from avod.core.models import avod_model

In [17]:
dataset=level5data
my_scene = dataset.scene[5]

        
voxel_size = (0.4,0.4,1.5)
z_offset = -2.0
#arbitrary shape, must be square though!
bev_shape = (224,224, 3)
        
# Only handle one sample at a time for now
my_sample_token = my_scene["first_sample_token"]
my_last_sample_token = my_scene["last_sample_token"]
sample = dataset.get('sample', my_sample_token)
tok=sample['data']["CAM_FRONT"]
boxes=dataset.get_boxes(tok)        

#anchors_info, obj_classes, label_classes, label_anchors, label_boxes_3d = preproc_helper.load_sample_info(sample_name, self.classes, self.dataset)

In [18]:
# Network input data: loop to get batch info
import box_helper
import bev_helper
import io
from typing import Tuple, List

classes=[]
for i in dataset.category:
    classes.append(i.get("name"))

img_input_list=[]
img_target=[]
bev_input_list=[]
ground_plane_list=[]

with tf.device('/GPU:0'):
    while my_sample_token!=my_last_sample_token:
        target_im = np.zeros([1024,1224,3], dtype=np.uint8)

        sample = dataset.get('sample', my_sample_token)
        sample_name = sample.get("token")
        img_data = dataset.get('sample_data', sample['data']["CAM_FRONT"])
        camera_token=img_data.get("token")

        tok=sample['data']["CAM_FRONT"]
        boxes=dataset.get_boxes(tok) 

        ego_pose = dataset.get("ego_pose", img_data["ego_pose_token"])
        data_path, boxes, camera_intrinsic = dataset.get_sample_data(camera_token)
        data = Image.open(data_path)

        anchors_info, obj_classes, label_classes, label_anchors, label_boxes_3d = preproc_helper.load_sample_info(sample_name, classes, dataset)
    #     fig, ax = plt.subplots(1, 1, figsize=(9, 16))

    #     # Show image.
    #     ax.imshow(data)

    #     # Show boxes.
    #     for box in boxes:
    #         c = np.array(box_helper.get_color(box.name)) / 255.0
    #         box.render(ax, view=camera_intrinsic, normalize=True, colors=(c, c, c))
    #     ax.axis('off')
    #     ax.set_title(img_data['channel'])
    #     ax.set_aspect('equal')

    #     # you can get a high-resolution image as numpy array!!
    #     plot_img_np = box_helper.get_img_from_fig(fig)


    #     # Limit visible range.
    #     ax.set_xlim(0, data.size[0])
    #     ax.set_ylim(data.size[1], 0)

    #    img_target.append(plot_img_np)
        bev_input = dataset.get('sample_data', sample['data']["LIDAR_TOP"])
        bev_data = bev_input
        bev_token= bev_input.get("token")
        lidar_data = dataset.get("sample_data", bev_token)
        lidar_filepath = dataset.get_sample_data_path(bev_token)
        ego_pose = dataset.get("ego_pose", lidar_data["ego_pose_token"])
        calibrated_sensor_lidar = dataset.get("calibrated_sensor", lidar_data["calibrated_sensor_token"])
        global_from_car = transform_matrix(ego_pose['translation'], Quaternion(ego_pose['rotation']), inverse=False)
        car_from_sensor_lidar = transform_matrix(calibrated_sensor_lidar['translation'], Quaternion(calibrated_sensor_lidar['rotation']),
                                                  inverse=False)
        lidar_pointcloud = LidarPointCloud.from_file(lidar_filepath)
        lidar_pointcloud.transform(car_from_sensor_lidar)
        map_mask = level5data.map[0]["mask"]
        bev = bev_helper.create_voxel_pointcloud(lidar_pointcloud.points, bev_shape, voxel_size=voxel_size, z_offset=z_offset)
        ego_centric_map = bev_helper.get_semantic_map_around_ego(map_mask, ego_pose, voxel_size=0.4, output_shape=(336,336)) 
        bev_array = bev_helper.normalize_voxel_intensities(bev)
        bev_input_list.append(bev_array)

        file_name=dataset.get_sample_data_path(camera_token)
        image = Image.open(file_name)
        # compress image, it is too big
        # convert image to numpy array
        img_array = np.asarray(image)
        if img_input_list is not None:
            img_input_list.append(img_array)

        cam_front_token = dataset.get('sample_data', sample['data']["CAM_FRONT"])
        cam_front_data = cam_front_token.get("calibrated_sensor_token")
        cam_front_calib = dataset.get("calibrated_sensor", cam_front_data )
        cam_front_coords = cam_front_calib.get("translation")

        cam_front_left_token = dataset.get('sample_data', sample['data']["CAM_FRONT_LEFT"])
        cam_front_left_data = cam_front_left_token.get("calibrated_sensor_token")
        cam_front_left_calib = dataset.get("calibrated_sensor", cam_front_left_data )
        cam_front_left_coords = cam_front_left_calib.get("translation")

        cam_front_right_token = dataset.get('sample_data', sample['data']["CAM_FRONT_RIGHT"])
        cam_front_right_data = cam_front_right_token.get("calibrated_sensor_token")
        cam_front_right_calib = dataset.get("calibrated_sensor", cam_front_right_data )
        cam_front_right_coords = cam_front_right_calib.get("translation")

        ground_plane = frame_helper.get_ground_plane_coeff(cam_front_coords, cam_front_left_coords, cam_front_right_coords)
        ground_plane_list.append(ground_plane)
        
        token=img_data.get("calibrated_sensor_token") 
        stereo_calib_p2 = frame_helper.read_calibration(token, dataset)

        my_sample_token = dataset.get("sample", my_sample_token)["next"]



In [19]:
all_anchor_boxes_3d = []
anchors_ious = []
anchor_offsets = []
anchor_classes = []
_bev_anchors_norm=[]

area_extents = pipeline_config[3].kitti_utils_config.area_extents

_area_extents = np.reshape(area_extents, (3, 2))
_bev_extents = _area_extents[[0, 2]]

label_cluster_utils = LabelClusterUtils(dataset)
_cluster_sizes, _all_std = label_cluster_utils.get_clusters(5, dataset)
_anchor_generator = grid_anchor_3d_generator.GridAnchor3dGenerator()

# Create anchors for each class
if len(classes) > 1:
    for class_idx in range(len(classes)):
        cluster_sizes = []
        for i in _cluster_sizes[class_idx]:
            if i!=[]:
                cluster_sizes.append(i)
        if len(cluster_sizes)!=0:
        # Generate anchors for all classes
            grid_anchor_boxes_3d = _anchor_generator.generate(area_3d=_area_extents, anchor_3d_sizes=cluster_sizes, anchor_stride=[0.5,0.5], ground_plane=ground_plane_list[0])
        else:
            #no labels per class, no anchor per class
            grid_anchor_boxes_3d=[]
        all_anchor_boxes_3d.append(grid_anchor_boxes_3d)

    all_anchor_boxes_3d = np.concatenate(all_anchor_boxes_3d, axis=None)
else:
    # Don't loop for a single class
    class_idx = 0
    cluster_sizes[class_idx] = [x for x in _cluster_sizes[class_idx] if x != []]
    if _cluster_sizes[class_idx]!=[]:
        grid_anchor_boxes_3d = _anchor_generator.generate(area_3d=_area_extents, anchor_3d_sizes=cluster_sizes[class_idx], 
                                                               anchor_stride=0.5, ground_plane=ground_plane_list[0])
        all_anchor_boxes_3d = grid_anchor_boxes_3d

# Filter empty anchors
# Skip if anchors_info is []
sample_has_labels = True

# Convert lists to ndarrays
#already filtered them before
anchor_boxes_3d_to_use = all_anchor_boxes_3d
anchors_ious = np.asarray(anchors_ious)
anchor_offsets = np.asarray(anchor_offsets)
anchor_classes = np.asarray(anchor_classes)

# Convert to anchors
anchors_to_use = box_3d_encoder.box_3d_to_anchor(anchor_boxes_3d_to_use)
num_anchors = len(anchors_to_use)

# Project anchors into bev
bev_anchors, bev_anchors_norm = anchor_projector.project_to_bev( anchors_to_use, _bev_extents)

# Reorder into [y1, x1, y2, x2] for tf.crop_and_resize op
_bev_anchors_norm.append(bev_anchors_norm[:, [1, 0, 3, 2]])

Clustering labels 126 / 126
Finished reading labels, clustering data...


Finished 



C:\Users\Utente1\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


In [20]:
img_input=np.asarray(img_input_list)
bev_input=np.asarray(bev_input_list)
#img_targetnp=np.asarray(img_target)

In [21]:
with tf.device('/CPU:0'):
    #inputs_vgg = tf.keras.layers.Input(shape=(224,224,3))
    vgg19 = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
    bev_data = tf.keras.applications.vgg19.preprocess_input(bev_input)
    bev_features=vgg19.predict(bev_data)
    #exclude fully connected
    output = vgg19.layers[-3].output
    #struggle with memory
    #img_data = tf.keras.applications.vgg19.preprocess_input(img_input)
    #img_features=vgg19.predict(img_data)

In [ ]:
import tensorflow as tf
random_values = tf.random.uniform(shape=[3], minval=0.0, maxval=1.0)
img_mask, bev_mask = create_path_drop_masks(0.5, 0.5, random_values)
bev_proposal_input = tf.multiply(bev_proposal_input, bev_mask)
bev_path_drop_mask = bev_mask

    # Overwrite the division factor
fusion_mean_div_factor = img_mask + bev_mask
print("\nStarting ROI pooling..\n")
def get_box_indices(boxes):
    proposals_shape = boxes.get_shape().as_list()
    if any(dim is None for dim in proposals_shape):
        proposals_shape = tf.shape(boxes)
    ones_mat = tf.ones(proposals_shape[:2], dtype=tf.int32)
    multiplier = tf.expand_dims(tf.range(start=0, limit=proposals_shape[0]), 1)
    return tf.reshape(ones_mat * multiplier, [-1])

# bev_boxes_norm_batches = tf.expand_dims(_bev_anchors_norm, axis=0)
# tf_box_indices = get_box_indices(bev_boxes_norm_batches)

# proposal_roi_size_tf = [3,3]
# # Do ROI Pooling on BEV eager ex on _bev_ancho
# bev_proposal_rois = tf.image.crop_and_resize(bev_proposal_input,
#                                              _bev_anchors_norm, tf_box_indices, proposal_roi_size_tf)

In [6]:
def create_path_drop_masks(self,
                               p_img,
                               p_bev,
                               random_values):
        """Determines global path drop decision based on given probabilities.

        Args:
            p_img: A tensor of float32, probability of keeping image branch
            p_bev: A tensor of float32, probability of keeping bev branch
            random_values: A tensor of float32 of shape [3], the results
                of coin flips, values should range from 0.0 - 1.0.

        Returns:
            final_img_mask: A constant tensor mask containing either one or zero
                depending on the final coin flip probability.
            final_bev_mask: A constant tensor mask containing either one or zero
                depending on the final coin flip probability.
        """

        def keep_branch(): return tf.constant(1.0)

        def kill_branch(): return tf.constant(0.0)

        # The logic works as follows:
        # We have flipped 3 coins, first determines the chance of keeping
        # the image branch, second determines keeping bev branch, the third
        # makes the final decision in the case where both branches were killed
        # off, otherwise the initial img and bev chances are kept.

        img_chances = tf.case([(tf.less(random_values[0], p_img), keep_branch)], default=kill_branch)

        bev_chances = tf.case([(tf.less(random_values[1], p_bev), keep_branch)], default=kill_branch)

        # Decision to determine whether both branches were killed off
        third_flip = tf.logical_or(tf.cast(img_chances, dtype=tf.bool), tf.cast(bev_chances, dtype=tf.bool))
        third_flip = tf.cast(third_flip, dtype=tf.float32)

        # Make a second choice, for the third case
        # Here we use a 50/50 chance to keep either image or bev
        # If its greater than 0.5, keep the image
        img_second_flip = tf.case([(tf.greater(random_values[2], 0.5), keep_branch)], default=kill_branch)
        # If its less than or equal to 0.5, keep bev
        bev_second_flip = tf.case([(tf.less_equal(random_values[2], 0.5), keep_branch)],
                                  default=kill_branch)

        # Use lambda since this returns another condition and it needs to
        # be callable
        final_img_mask = tf.case([(tf.equal(third_flip, 1), lambda: img_chances)], default=lambda: img_second_flip)

        final_bev_mask = tf.case([(tf.equal(third_flip, 1), lambda: bev_chances)], default=lambda: bev_second_flip)

        return final_img_mask, final_bev_mask